# Country Names Map

This loads a GeoJSON with all country borders which are then shown on a map according to their names starting with a certain letter to be chosen with a slider in the corner of the map.

In [ ]:
from functools import partial
from string import ascii_uppercase

from ipywidgets import IntSlider, Layout
from ipyleaflet import basemaps, Map, GeoJSON, LayersControl, FullScreenControl, WidgetControl

from xyzspaces.datasets import get_countries_data

In [ ]:
def add_countries(a_map, features):
    "Add a list of GeoJSON objects to a map."
    other_layers = [l for l in a_map.layers if type(l) != GeoJSON]
    a_map.layers = tuple(other_layers)
    for f in features:
        gj_layer = GeoJSON(data=f, name=f["properties"]["name"])
        a_map.add_layer(gj_layer)

In [ ]:
def slider_moved(event, the_map=None, the_features=None):
    "Callback for slider events."
    if event.type != 'change' or type(event.new) != int:
        return
    n = event.new
    assert 0 <= n < 26
    ch = ascii_uppercase[n]
    features = [f for f in the_features["features"] 
        if f["properties"]["name"].upper().startswith(ch)
        # if ch in f["properties"]["name"].upper()
    ]
    event["owner"].description = f"{len(features)} x {ch}"
    add_countries(the_map, features)    
    names = [f["properties"]["name"] for f in features]
    return names

In [ ]:
countries = get_countries_data()

m = Map(zoom=2, basemap=basemaps.OpenStreetMap["Mapnik"])
m += FullScreenControl(position="topleft")
m += LayersControl(position="topright")

layout = Layout(width="500px")
letter_slider = IntSlider(min=0, max=25, step=1, value=1, description=" ", layout=layout)
letter_slider.observe(partial(slider_moved, the_map=m, the_features=countries))
widget_control1 = WidgetControl(widget=letter_slider, position='bottomleft', layout=layout)
m += widget_control1

# This is needed to trigger the first slider change event,
# or the map would be empty at the start:
letter_slider.value = 0

m